In [1]:
import pandas as pd
import os
import plotly.express as px

regions_complete = pd.read_csv("regions_complete.csv", sep=";")

In [2]:
# get the csv files from the size_points folder

size_points_path = "size_points_S1_injection"
min_pixel_size = 1
max_pixel_size = 17


# loop through the files and save the csv files as seperate dataframes with the name of the file
brains = {}
for file in os.listdir(size_points_path):
    if file.endswith(".csv"):
        # get the brain name from the file name
        brain_name = file.split("_")[0:6]
        # join the brain name parts with an underscore
        brain_name = "_".join(brain_name)
        # take away .csv from the end of the brain name
        brain_name = brain_name.replace(".csv", "")

        print(brain_name)
        brains[brain_name] = pd.read_csv(os.path.join(size_points_path, file), sep=";")

WT_n1_P6_Objects_All_black
WT_n1_P6_Objects_All_white
WT_n2_P6_Objects_All_black
WT_n2_P6_Objects_All_white
WT_n2_P7_Objects_All_black
WT_n2_P7_Objects_All_white


ipsilateral = injection site of hemisphere
contralateral = opposite side

In [3]:
# generate one file per brain


def end_file_per_brain(white, black, name, age, injection_side, injection_area):
    one_brain_file = pd.DataFrame(
        columns=[
            "Brain",
            "Region ID",
            "General area",
            "Hemisphere",
            "count",
            "scaled count",
            "age",
        ]
    )
    total_counts = 0
    if injection_side == "left":
        white_side = "contralateral"
        black_side = "ipsilateral"
    else:
        white_side = "ipsilateral"
        black_side = "contralateral"

    for index, row in white.iterrows():

        hemisphere = white_side

        if min_pixel_size <= row["Object area"] <= max_pixel_size:
            region_id = row["Region ID"]

            # check for the according General area in regions_complete
            general_region = regions_complete.loc[
                regions_complete["Region ID"] == region_id, "General area"
            ].values[0]
            subdivided = regions_complete.loc[
                regions_complete["Region ID"] == region_id, "Subdivided"
            ].values[0]

            if subdivided != injection_area:

                # if region_id is not in one_brain_file, add it
                if region_id not in one_brain_file["Region ID"].values:
                    one_brain_file.loc[region_id] = [
                        name,
                        region_id,
                        general_region,
                        hemisphere,
                        1,
                        0,
                        age,
                    ]
                else:
                    one_brain_file.loc[region_id, "count"] += 1
            if subdivided == injection_area and injection_side == "left":
                if region_id not in one_brain_file["Region ID"].values:
                    one_brain_file.loc[region_id] = [
                        name,
                        region_id,
                        general_region,
                        hemisphere,
                        1,
                        0,
                        age,
                    ]
                else:
                    one_brain_file.loc[region_id, "count"] += 1

            # the counts for clear label and fiber tracts are not counted
            if (
                general_region not in ["Clear Label", "fiber tracts", "Parent"]
                and subdivided != "injection_area"
            ):
                total_counts += 1

    for index, row in black.iterrows():
        hemisphere = black_side

        if min_pixel_size <= row["Object area"] <= max_pixel_size:
            region_id = row["Region ID"]

            # check for the according General area in regions_complete
            general_region = regions_complete.loc[
                regions_complete["Region ID"] == region_id, "General area"
            ].values[0]
            subdivided = regions_complete.loc[
                regions_complete["Region ID"] == region_id, "Subdivided"
            ].values[0]

            if subdivided != injection_area:

                # if region_id is not in one_brain_file with "right" in the hemisphere, add it
                if region_id not in one_brain_file["Region ID"].values:
                    one_brain_file.loc[region_id] = [
                        name,
                        region_id,
                        general_region,
                        hemisphere,
                        1,
                        0,
                        age,
                    ]

                else:
                    # if region_id is already in one_brain_file check if the hemisphere is opposite or same side
                    if one_brain_file.loc[region_id, "Hemisphere"] == black_side:
                        # if it is, add the count to the existing count
                        one_brain_file.loc[region_id, "count"] += 1
                    else:
                        one_brain_file.loc[region_id] = [
                            name,
                            region_id,
                            general_region,
                            hemisphere,
                            1,
                            0,
                            age,
                        ]

                if subdivided == injection_area and injection_side == "right":
                    if region_id not in one_brain_file["Region ID"].values:
                        one_brain_file.loc[region_id] = [
                            name,
                            region_id,
                            general_region,
                            hemisphere,
                            1,
                            0,
                            age,
                        ]
                    else:
                        one_brain_file.loc[region_id, "count"] += 1

            if (
                general_region not in ["Clear Label", "fiber tracts", "Parent"]
                and subdivided != injection_area
            ):
                total_counts += 1

    # scale the counts
    one_brain_file["scaled count"] = 100 / total_counts * one_brain_file["count"]

    return one_brain_file

In [4]:
WT_n1_P6 = end_file_per_brain(
    brains["WT_n1_P6_Objects_All_white"],
    brains["WT_n1_P6_Objects_All_black"],
    "WT_n1_P6",
    "p6",
    "left",
    "Primary somatosensory area",
)
WT_n2_P6 = end_file_per_brain(
    brains["WT_n2_P6_Objects_All_white"],
    brains["WT_n2_P6_Objects_All_black"],
    "WT_n2_P6",
    "p6",
    "left",
    "Primary somatosensory area",
)
WT_n2_P7 = end_file_per_brain(
    brains["WT_n2_P7_Objects_All_white"],
    brains["WT_n2_P7_Objects_All_black"],
    "WT_n2_P7",
    "p7",
    "left",
    "Primary somatosensory area",
)

# get into Subdivided to get out specifically the injection area

In [5]:
# make the big mega file with all the brains put the brain name in the first column
big_file_S1_injections = pd.concat(
    [
        WT_n1_P6,
        WT_n2_P6,
        WT_n2_P7,
    ],
    ignore_index=True,
)

In [6]:
# take out Clear label and fiber tracts
big_file_S1_injections = big_file_S1_injections[
    ~big_file_S1_injections["General area"].isin(
        ["Clear Label", "fiber tracts", "Parent"]
    )
]

In [7]:
# Find regions with zero neurons


def find_zero_neurons(
    big_file,
    general_area_ipsi_age_A,
    general_area_contra_age_A,
    general_area_ipsi_age_B,
    general_area_contra_age_B,
    age_A,
    age_B,
):

    # add General area that are in regions_complete but not in S1_injections
    regions_complete = pd.read_csv("regions_complete.csv", sep=";")
    General_areas_complete = regions_complete["General area"].unique()

    # filter out "Clear Label" and "Fiber tracts" and "Parent" from General areas
    General_areas_complete = [
        area
        for area in General_areas_complete
        if area not in ["Clear Label", "fiber tracts", "Parent", "retina"]
    ]

    # make copy of the DataFrame to add missing General areas
    big_file_long = big_file.copy()

    general_area_ipsi_age_A = []
    general_area_contra_age_A = []
    general_area_ipsi_age_B = []
    general_area_contra_age_B = []

    # loop through big_file_long and add General area to lists
    for index, row in big_file_long.iterrows():
        if row["Hemisphere"] == "ipsilateral":
            if row["age"] == age_A:
                general_area_ipsi_age_A.append(row["General area"])
            else:
                general_area_ipsi_age_B.append(row["General area"])

        else:
            if row["age"] == age_A:
                general_area_contra_age_A.append(row["General area"])
            else:
                general_area_contra_age_B.append(row["General area"])

    for area in General_areas_complete:
        if area not in general_area_ipsi_age_A:
            print("area not found in", "general_area_ipsi", age_A, ":", area)

            big_file_long.loc[area] = [0, 0, area, "ipsilateral", 0, 0, age_A]

        if area not in general_area_contra_age_A:
            print("area not found in", "general_area_contra", age_A, ":", area)
            big_file_long.loc[area] = [0, 0, area, "contralateral", 0, 0, age_A]

        if area not in general_area_ipsi_age_B:
            print("area not found in", "general_area_ipsi", age_B, ":", area)
            big_file_long.loc[area] = [0, 0, area, "ipsilateral", 0, 0, age_B]

        if area not in general_area_contra_age_B:
            print("area not found in", "general_area_contra", age_B, ":", area)
            big_file_long.loc[area] = [0, 0, area, "contralateral", 0, 0, age_B]

    return big_file_long

In [8]:
big_file_S1_injections_long = find_zero_neurons(
    big_file_S1_injections,
    "general_area_ipsi_p6",
    "general_area_contra_p6",
    "general_area_ipsi_p7",
    "general_area_contra_p7",
    "p6",
    "p7",
)

area not found in general_area_contra p6 : root
area not found in general_area_contra p7 : root
area not found in general_area_contra p6 : Frontal pole
area not found in general_area_contra p6 : Somatomotor areas
area not found in general_area_contra p7 : Somatomotor areas
area not found in general_area_contra p6 : Visceral area
area not found in general_area_contra p7 : Visceral area
area not found in general_area_contra p6 : Auditory areas
area not found in general_area_contra p7 : Auditory areas
area not found in general_area_contra p7 : Visual areas
area not found in general_area_contra p7 : Anterior cingulate area
area not found in general_area_contra p7 : Prelimbic area
area not found in general_area_contra p7 : Infralimbic area
area not found in general_area_contra p6 : Retrosplenial area
area not found in general_area_contra p7 : Retrosplenial area
area not found in general_area_contra p6 : Posterior parietal association areas
area not found in general_area_contra p7 : Posterio

In [9]:
# save the big file as csv
big_file_S1_injections_long.to_csv(
    "big_file_S1_injections_long.csv", sep=";", index=False
)

In [10]:
big_file_S1_injections_long

,Brain,Region ID,General area,Hemisphere,count,scaled count,age
1,WT_n1_P6,228,Main olfactory bulb,ipsilateral,31,0.417508,p6
2,WT_n1_P6,244,Main olfactory bulb,ipsilateral,94,1.265993,p6
3,WT_n1_P6,236,Main olfactory bulb,ipsilateral,65,0.875421,p6
4,WT_n1_P6,220,Main olfactory bulb,ipsilateral,32,0.430976,p6
5,WT_n1_P6,507,Main olfactory bulb,ipsilateral,30,0.404040,p6
...,...,...,...,...,...,...,...
Retrohippocampal region,0,0,Retrohippocampal region,contralateral,0,0.000000,p7
Striatum,0,0,Striatum,contralateral,0,0.000000,p7
Pons,0,0,Pons,contralateral,0,0.000000,p7
Medulla,0,0,Medulla,contralateral,0,0.000000,p7


In [11]:
# make a file with only general regions counts which are then scaled to the total count of the brain
general_areas_file = pd.DataFrame(
    columns=[
        "Brain",
        "General area",
        "Hemisphere",
        "count",
        "scaled count",
        "age",
    ]
)
totals = pd.DataFrame(columns=["Brain", "total"])
for index, row in big_file_S1_injections_long.iterrows():
    general_area = row["General area"]
    hemisphere = row["Hemisphere"]
    age = row["age"]
    name = row["Brain"]
    amount = row["count"]

    if name not in totals["Brain"].values:
        totals.loc[name] = [name, amount]
    else:
        totals.loc[name, "total"] += amount

In [13]:
df_filtered = big_file_S1_injections_long[
    ["Brain", "age", "count", "Hemisphere", "General area"]
]

# Step 2: Group by the relevant identifiers and sum the count per group
df_grouped = df_filtered.groupby(
    ["Brain", "General area", "Hemisphere", "age"], as_index=False
)["count"].sum()

# add scaled column with 0
df_grouped["scaled count"] = 0

# scale the counts
for index, row in df_grouped.iterrows():
    brain_name = row["Brain"]
    count = row["count"]
    total_count = totals.loc[totals["Brain"] == brain_name, "total"].values[0]
    scaled_count = 1 / total_count * count
    df_grouped.loc[index, "scaled count"] = scaled_count
df_grouped

C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\191345882.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  scaled_count = 1 / total_count * count
C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\191345882.py:18: RuntimeWarning: invalid value encountered in scalar multiply
  scaled_count = 1 / total_count * count
C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\191345882.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  scaled_count = 1 / total_count * count
C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\191345882.py:18: RuntimeWarning: invalid value encountered in scalar multiply
  scaled_count = 1 / total_count * count
C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\191345882.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  scaled_count = 1 / total_count * count
C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\191345882.py:18: RuntimeWarning: invalid value encountered in scalar multiply
  scaled_count = 1 / total_count * c

,Brain,General area,Hemisphere,age,count,scaled count
0,0,Accessory olfactory bulb,contralateral,p7,0,NaN
1,0,Anterior cingulate area,contralateral,p7,0,NaN
2,0,Anterior olfactory nucleus,contralateral,p7,0,NaN
3,0,Auditory areas,contralateral,p7,0,NaN
4,0,Cerebellum,contralateral,p7,0,NaN
...,...,...,...,...,...,...
173,WT_n2_P7,Visceral area,ipsilateral,p7,29,0.004232
174,WT_n2_P7,Visual areas,ipsilateral,p7,1933,0.282066
175,WT_n2_P7,root,ipsilateral,p7,35,0.005107
176,WT_n2_P7,ventricular systems,contralateral,p7,1,0.000146


In [14]:
def get_area_order_by_age(big_file_long):
    # Filter for 'ipsilateral' hemisphere and age group 'p7-13'
    filtered = big_file_long[
        (big_file_long["Hemisphere"] == "ipsilateral")
        & (big_file_long["age"] == "p7-13")
    ]

    area_order_scaled = (
        filtered.groupby("General area")["scaled count"]
        .max()
        .sort_values(ascending=False)
        .index.tolist()
    )

    area_order_count = (
        filtered.groupby("General area")["count"]
        .max()
        .sort_values(ascending=False)
        .index.tolist()
    )

    return area_order_scaled, area_order_count

In [22]:
area_order_scaled, area_order_count = get_area_order_by_age(df_grouped)

In [101]:
def plot_count(big_file_long, Title, area_order_count):


    fig = px.violin(big_file_long,
        y="count",
        x="General area",
        color="Hemisphere",
        box=True,
        points="all",
        color_discrete_map={
            "contralateral": "orange",
            "ipsilateral": "magenta"
        },

    title=Title,
    category_orders={"General area": area_order_count}
   
    
    )
    fig.update_traces(meanline_visible=True)    # displays the arithmetic mean of the distribution
    fig.update_layout(violinmode= "group") #group left(right violins side-by-side)
    

    return fig


    

In [53]:
def plot_scaled(big_file_long, Title, area_order_scaled):


    fig = px.violin(big_file_long,
        y="scaled count",
        x="General area",
        color="Hemisphere",
        box=True,
        points="all",
        color_discrete_map={
            "contralateral": "orange",
            "ipsilateral": "magenta"
        },

    title=Title,
    category_orders={"General area": area_order_scaled}
   
    
    )
    fig.update_traces(meanline_visible=True)    # displays the arithmetic mean of the distribution
    fig.update_layout(violinmode= "group") #group left(right violins side-by-side)
    

    return fig


    

In [102]:
plot_count(big_file_S1_injections_long, "S1 injections counts by hemisphere, both p6 (n=2) and p7 (n=1)", area_order_count)

In [103]:
plot_scaled(big_file_S1_injections_long, "S1 injections scaled counts by hemisphere, both p6 (n=2) and p7 (n=1)", area_order_scaled)

In [41]:
def plot_scaled_zoomed(big_file_long, Title):
    filtered_areas = [
        "Visual areas",
        "Retrosplenial area",
        "Thalamus",
        "Main olfactory bulb",
        "Piriform area",
        "Auditory areas",
    ]

    df_hemi = big_file_long[
        (big_file_long["Hemisphere"] == "ipsilateral")
        & (big_file_long["General area"].isin(filtered_areas))
    ]

    area_order_scaled = (
        df_hemi.groupby("General area")["scaled count"]
        .max()
        .sort_values(ascending=False)
        .index.tolist()
    )

    fig = px.violin(
        df_hemi,
        y="scaled count",
        x="General area",
        # color="age",
        box=True,
        # points="all",
        title=Title,
        category_orders={"General area": area_order_scaled},
    )
    fig.update_traces(
        meanline_visible=True
    )  # displays the arithmetic mean of the distribution

    return fig

In [42]:
def plot_count_zoomed(big_file_long, Title):
    filtered_areas = [
        "Visual areas",
        "Retrosplenial area",
        "Thalamus",
        "Main olfactory bulb",
        "Piriform area",
        "Auditory areas",
    ]

    df_hemi = big_file_long[
        (big_file_long["Hemisphere"] == "ipsilateral")
        & (big_file_long["General area"].isin(filtered_areas))
    ]

    area_order_count = (
        df_hemi.groupby("General area")["count"]
        .max()
        .sort_values(ascending=False)
        .index.tolist()
    )

    fig = px.violin(
        df_hemi,
        y="count",
        x="General area",
        # color="age",
        box=True,
        # points="all",
        title=Title,
        category_orders={"General area": area_order_count},
    )
    fig.update_traces(
        meanline_visible=True
    )  # displays the arithmetic mean of the distribution

    return fig

In [43]:
plot_scaled_zoomed(
    df_grouped,
    "S1 injections scaled counts only ipsilateral, p7(n=1) and p6 (n=2?)",
)

In [44]:
plot_count_zoomed(
    df_grouped,
    "S1 injections count only ipsilateral, p7(n=1) and p6 (n=2?)",
)

In [45]:
def specific_interests(big_file_long, area_name, data_frame_name):

    data_frame_name = pd.DataFrame(
        big_file_long[big_file_long["General area"].isin([area_name])]
    )

    data_frame_name.insert(7, "Subdivided", [0] * len(data_frame_name))

    for index, row in data_frame_name.iterrows():
        region_id = row["Region ID"]
        subdivided_region = regions_complete.loc[
            regions_complete["Region ID"] == region_id, "Subdivided"
        ].values[0]
        # add the Subdivided values to the Somatosensory_areas DataFrame
        data_frame_name.at[index, "Subdivided"] = subdivided_region

    return data_frame_name

area 29 (granular): ventral RSC
area 30 (dysgranular): dorsal RSC and agranular RSC

In [46]:
Retrosplenial_areas = specific_interests(
    big_file_S1_injections_long, "Retrosplenial area", "Retrosplenial_areas"
)
Visual_areas = specific_interests(
    big_file_S1_injections_long, "Visual areas", "Visual_areas"
)

C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\2296359539.py:15: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'area 30' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\2296359539.py:15: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Anteromedial visual area' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [47]:
subdivided_rsc_filtered = Retrosplenial_areas[
    ["Brain", "age", "count", "Hemisphere", "Subdivided"]
]

subdivided_rsc_grouped = subdivided_rsc_filtered.groupby(
    ["Brain", "Subdivided", "Hemisphere", "age"], as_index=False
)["count"].sum()

# add scaled column with 0
subdivided_rsc_grouped["scaled count"] = 0
# scale the counts
for index, row in subdivided_rsc_grouped.iterrows():
    brain_name = row["Brain"]
    count = row["count"]
    total_count = totals.loc[totals["Brain"] == brain_name, "total"].values[0]
    scaled_count = 1 / total_count * count
    subdivided_rsc_grouped.loc[index, "scaled count"] = scaled_count
subdivided_rsc_grouped

C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\844754293.py:17: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.023129685755303876' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



,Brain,Subdivided,Hemisphere,age,count,scaled count
0,WT_n1_P6,area 29,ipsilateral,p6,145,0.023130
1,WT_n1_P6,area 30,ipsilateral,p6,561,0.089488
2,WT_n2_P6,area 29,ipsilateral,p6,915,0.069890
3,WT_n2_P6,area 30,ipsilateral,p6,2043,0.156049
4,WT_n2_P7,area 29,ipsilateral,p7,382,0.055742
5,WT_n2_P7,area 30,ipsilateral,p7,583,0.085072


In [52]:
subdivided_Vis_filtered = Visual_areas[
    ["Brain", "age", "count", "Hemisphere", "Subdivided"]
]

subdivided_Vis_grouped = subdivided_Vis_filtered.groupby(
    ["Brain", "Subdivided", "Hemisphere", "age"], as_index=False
)["count"].sum()

# add scaled column with 0
subdivided_Vis_grouped["scaled count"] = 0
# scale the counts
for index, row in subdivided_Vis_grouped.iterrows():
    brain_name = row["Brain"]
    count = row["count"]
    total_count = totals.loc[totals["Brain"] == brain_name, "total"].values[0]
    scaled_count = 1 / total_count * count
    subdivided_Vis_grouped.loc[index, "scaled count"] = scaled_count
subdivided_Vis_grouped

C:\Users\sahet\AppData\Local\Temp\ipykernel_6104\3134058521.py:17: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01260169085978625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



,Brain,Subdivided,Hemisphere,age,count,scaled count
0,WT_n1_P6,Anterolateral visual area,ipsilateral,p6,79,0.012602
1,WT_n1_P6,Anteromedial visual area,ipsilateral,p6,175,0.027915
2,WT_n1_P6,Lateral visual area,ipsilateral,p6,38,0.006062
3,WT_n1_P6,Laterointermediate area,contralateral,p6,1,0.000160
4,WT_n1_P6,Laterointermediate area,ipsilateral,p6,23,0.003669
5,WT_n1_P6,Postrhinal area,ipsilateral,p6,17,0.002712
6,WT_n1_P6,Primary visual area,ipsilateral,p6,664,0.105918
7,WT_n1_P6,posteromedial visual area,ipsilateral,p6,190,0.030308
8,WT_n2_P6,Anterolateral visual area,ipsilateral,p6,259,0.019783
9,WT_n2_P6,Anteromedial visual area,ipsilateral,p6,480,0.036664


In [57]:
def plot_count_zoomed_sub(big_file_long, Title):

    df_hemi = big_file_long[big_file_long["Hemisphere"] == "ipsilateral"]
    area_order_count = (
        df_hemi.groupby("Subdivided")["count"]
        .max()
        .sort_values(ascending=False)
        .index.tolist()
    )

    fig = px.violin(
        df_hemi,
        y="count",
        x="Subdivided",
        box=True,
        title=Title,
        category_orders={"Subdivided": area_order_count},
    )
    fig.update_traces(
        meanline_visible=True
    )  # displays the arithmetic mean of the distribution
    fig.update_layout(violinmode="group")  # group left(right violins side-by-side)

    return fig

In [58]:
plot_count_zoomed_sub(
    subdivided_rsc_grouped,
    "S1 injections counts in Retrosplenial area subdivisions ipsilateral only",
)

In [65]:
def plot_scaled_count_zoomed_sub(big_file_long, Title):

    df_hemi = big_file_long[big_file_long["Hemisphere"] == "ipsilateral"]
    area_order_scaled_count = (
        df_hemi.groupby("Subdivided")["scaled count"]
        .max()
        .sort_values(ascending=False)
        .index.tolist()
    )

    fig = px.violin(
        df_hemi,
        y="scaled count",
        x="Subdivided",
        box=True,
        title=Title,
        category_orders={"Subdivided": area_order_scaled_count},
    )
    fig.update_traces(
        meanline_visible=True
    )  # displays the arithmetic mean of the distribution
    fig.update_layout(violinmode="group")  # group left(right violins side-by-side)

    return fig

In [66]:
plot_scaled_count_zoomed_sub(
    subdivided_rsc_grouped,
    "Retrosplenial area scaled counts by hemisphere, both p6 (n=2) and p7 (n=1)",
)

In [61]:
plot_count_zoomed_sub(
    subdivided_Vis_grouped,
    "Visual area counts by hemisphere, both p6 (n=2) and p7 (n=1)",
)

In [62]:
plot_scaled_count_zoomed_sub(
    subdivided_Vis_grouped,
    "Visual area scaled counts by hemisphere, both p6 (n=2) and p7 (n=1)",
)